In [2]:
import numpy as np 
import pandas as pd
import datetime
import calendar
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', None)
%matplotlib inline

In [3]:
X_train = pd.read_csv('../data/X_train.csv', sep=',', encoding='utf-8')
target = pd.read_csv('../data/target.csv', sep=',', encoding='utf-8')
X_test = pd.read_csv('../data/X_test.csv', sep=',', encoding='utf-8')

In [61]:
print('train shape:', X_train.shape,'| target shape:', target.shape)
print('test shape:', X_test.shape)

train shape: (12815883, 13) | target shape: (12815883, 1)
test shape: (104530, 14)


In [4]:
X_test.duplicated('id').any()

False

In [5]:
ids = X_test[['id']]
X_test.drop('id', axis=1, inplace=True)

In [6]:
ids.to_csv('test_.csv', sep=',', encoding='utf-8')

In [7]:
X_test.shape

(100000, 13)

In [8]:
print(len(target[target['postulado'] == False]))
print(len(target[target['postulado'] == True]))

6497662
6318221


## TEMPORAL: se eliminan las columnas con los arrays, decidir que hacer con este dato

In [9]:
X_train.drop(columns=['descripcion', 'titulo'], inplace=True)
X_test.drop(columns=['descripcion', 'titulo'], inplace=True)

In [20]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [21]:
X_train.head(1)

,edad,estado_estudio,idaviso,idpostulante,nivel_laboral,nombre_area,nombre_estudio,sexo,tipo_full,tipo_otro,tipo_part
0,32,3,12543760,ek43a69,Senior / Semi-Senior,Diseño Gráfico,4,FEM,True,False,False


In [ ]:
# TODO!
#print('train: ',label_encoder.fit(X_train.nivel_laboral).classes_,'->',np.unique(label_encoder.transform(X_train.nivel_laboral)))
#X_train.nivel_laboral = label_encoder.fit_transform(X_train.nivel_laboral)
#print('test:  ',label_encoder.fit(X_test.nivel_laboral).classes_,'->',np.unique(label_encoder.transform(X_test.nivel_laboral)))
#X_test.nivel_laboral = label_encoder.fit_transform(X_test.nivel_laboral)

In [10]:
# Solo para probar un submit elimino las columnas que no son numericas/booleanas
X_train.drop(columns=['nivel_laboral', 'nombre_area', 'sexo'], inplace=True)
X_test.drop(columns=['nivel_laboral', 'nombre_area', 'sexo'], inplace=True)

In [11]:
X_train.drop(columns=['idpostulante'], inplace=True)
X_test.drop(columns=['idpostulante'], inplace=True)

In [12]:
print(X_train.isnull().values.any())
print(X_test.isnull().values.any())

False
True


In [40]:
X_test.head(1)

,idaviso,nombre_estudio,estado_estudio,edad,tipo_full,tipo_part,tipo_otro
0,739260,7.0,3.0,42.0,True,False,False


In [13]:
X_test['edad'].isnull().any()

True

In [ ]:
X_test

In [14]:
moda_nombre_estudio = X_test.nombre_estudio.mode()[0]
moda_estado_estudio = X_test.estado_estudio.mode()[0]
media_edad = X_test.edad.mode()[0]
X_test.nombre_estudio.fillna(moda_nombre_estudio,inplace=True)
X_test.estado_estudio.fillna(moda_estado_estudio,inplace=True)
X_test.edad.fillna(media_edad,inplace=True)

In [18]:
from sklearn import tree
from sklearn.linear_model import SGDClassifier

In [19]:
def predict(model,X_train,X_test,y):
    model.fit(X_train,y)
    return model.predict(X_test)

def pred_to_csv(prediction,name):
    test = pd.read_csv('test_.csv')
    df = pd.DataFrame(prediction,columns={'sepostulo'})
    df['id'] = test.id
    df = df.loc[:,['id','sepostulo']]
    df.sepostulo = df.sepostulo.astype(int)
    df.to_csv('pred_'+name+'.csv',index=False)

In [20]:
def predict(model,X_train,X_test,y):
    model.fit(X_train,y)
    return model.predict(X_test)

In [21]:
pred = predict(SGDClassifier(),X_train,X_test,target)
pred_to_csv(pred,'SGD')

/home/tbotalla/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/tbotalla/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
from sklearn.model_selection import cross_val_score, KFold
# seed para generar los mismos fold en cada cv
seed = 7
kfold = KFold(10,random_state=seed)
scoring = 'neg_mean_squared_error'
#scoring = 'jaccard_similarity_score'
def cv_score(model,X,y,cv):
#    MSE_score = cross_val_score(model,X,y,n_jobs=-1,cv=cv,scoring='neg_mean_squared_error').mean()
    return cross_val_score(model,X,y,n_jobs=-1,cv=cv,scoring=scoring,verbose=3).mean()
#    print('params:\n',model.get_params())
#    print('MSE score:\n',MSE_score)

In [ ]:
from sklearn.preprocessing import scale, normalize
X_train_std = scale(X_train)
X_test_std = scale(X_test)
#X_train_nrm = normalize(X_train)
#X_test_nrm = normalize(X_test)

In [ ]:
%%time
models = []
models.append(('DecTree',tree.DecisionTreeClassifier()))
scores = []
for name,model in models:
    print('################################',name,'################################')
    score = cv_score(model,X_train,target,cv=kfold)
    scores.append((name,score))

################################ DecTree ################################
